# Building a ChatBot with a Seq2Seq LSTM model

![alt text](https://miro.medium.com/max/2142/0*LL_VfEsZQ1wW1391.png)

We'll build a Seq2Seq model just like this one, but with LSTM cells instead of GRU cells. We'll use GloVe embeddings to build a chatbot using an open-source Kaggle dataset.

## Dataset

We'll use the dataset from the source below.  It contains pairs of questions and answers based on a number of subjects like food, history, AI etc.

[ChatterBot Language Training Corpus](https://github.com/gunthercox/chatterbot-corpus)

In [ ]:
import requests, zipfile, io

r = requests.get( 'https://github.com/shubham0204/Dataset_Archives/blob/master/chatbot_nlp.zip?raw=true' ) 
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall()

Reading the data from the files:

We parse each of the `.yaml` files.

*   Concatenate two or more sentences if the answer has two or more of them
*   Remove unwanted data types which are produced while parsing the data
*   Append `<START>` and `<END>` to all the `answers`
*   Create a `Tokenizer` and load the whole vocabulary ( `questions` + `answers` )

In [3]:

import numpy as np
import tensorflow as tf
import pickle
from tensorflow.keras import layers , activations , models , preprocessing

print( tf.VERSION )

1.15.0


In [4]:
from tensorflow.keras import preprocessing , utils
import os
import yaml

dir_path = 'chatbot_nlp/data'
files_list = os.listdir(dir_path + os.sep)

questions = list()
answers = list()

for filepath in files_list:
    stream = open( dir_path + os.sep + filepath , 'rb')
    docs = yaml.safe_load(stream)
    conversations = docs['conversations']
    for con in conversations:
        if len( con ) > 2 :
            questions.append(con[0])
            replies = con[ 1 : ]
            ans = ''
            for rep in replies:
                ans += ' ' + rep
            answers.append( ans )
        elif len( con )> 1:
            questions.append(con[0])
            answers.append(con[1])

answers_with_tags = list()
for i in range( len( answers ) ):
    if type( answers[i] ) == str:
        answers_with_tags.append( answers[i] )
    else:
        questions.pop( i )

answers = list()
for i in range( len( answers_with_tags ) ) :
    answers.append( '<START> ' + answers_with_tags[i] + ' <END>' )

tokenizer = preprocessing.text.Tokenizer()
tokenizer.fit_on_texts( questions + answers )
VOCAB_SIZE = len( tokenizer.word_index )+1
print( 'VOCAB SIZE : {}'.format( VOCAB_SIZE ))

VOCAB SIZE : 1894


## Pre-processing Data

Our model requires three arrays namely 

1) `encoder_input_data`: Tokenize the questions and pad them to their max length

2) `decoder_input_data` : Tokenize the answers and pad them to their max length

3) `decoder_output_data` : Tokenize the answers and remove the first element from all the tokenized_answers = This is the `<START>` element which we added earlier



In [5]:

from gensim.models import Word2Vec
import re

vocab = []
for word in tokenizer.word_index:
    vocab.append(word)

def tokenize(sentences):
    tokens_list = []
    vocabulary = []
    for sentence in sentences:
        sentence = sentence.lower()
        sentence = re.sub( '[^a-zA-Z]', ' ', sentence )
        tokens = sentence.split()
        vocabulary += tokens
        tokens_list.append(tokens)
    return tokens_list , vocabulary

p = tokenize(questions + answers)
model = Word2Vec(p[ 0 ]) 

embedding_matrix = np.zeros((VOCAB_SIZE , 100))
for i in range(len(tokenizer.word_index)):
  try:
    embedding_matrix[i] = model[vocab[i]]
  except KeyError:
    continue 

# encoder_input_data
tokenized_questions = tokenizer.texts_to_sequences(questions)
maxlen_questions = max([len(x) for x in tokenized_questions])
padded_questions = preprocessing.sequence.pad_sequences(tokenized_questions , maxlen=maxlen_questions , padding='post')
encoder_input_data = np.array( padded_questions )
print( encoder_input_data.shape , maxlen_questions )

# decoder_input_data
tokenized_answers = tokenizer.texts_to_sequences( answers )
maxlen_answers = max( [ len(x) for x in tokenized_answers ] )
padded_answers = preprocessing.sequence.pad_sequences(tokenized_answers , maxlen=maxlen_answers , padding='post')
decoder_input_data = np.array( padded_answers )
print( decoder_input_data.shape , maxlen_answers )

# decoder_output_data
tokenized_answers = tokenizer.texts_to_sequences( answers )
for i in range(len(tokenized_answers)) :
    tokenized_answers[i] = tokenized_answers[i][1:]
padded_answers = preprocessing.sequence.pad_sequences(tokenized_answers , maxlen=maxlen_answers , padding='post')
onehot_answers = utils.to_categorical(padded_answers , VOCAB_SIZE)
decoder_output_data = np.array(onehot_answers)
print( decoder_output_data.shape)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:26: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


(564, 22) 22
(564, 74) 74
(564, 74, 1894)


### Model Architecture A:

In [6]:
encoder_inputs = tf.keras.layers.Input(shape=( None , ))
encoder_embedding = tf.keras.layers.Embedding( VOCAB_SIZE, 200 , mask_zero=True ) (encoder_inputs)
encoder_outputs , state_h , state_c = tf.keras.layers.LSTM( 200 , return_state=True )( encoder_embedding )
encoder_states = [ state_h , state_c ]

decoder_inputs = tf.keras.layers.Input(shape=( None ,  ))
decoder_embedding = tf.keras.layers.Embedding( VOCAB_SIZE, 200 , mask_zero=True) (decoder_inputs)
decoder_lstm = tf.keras.layers.LSTM( 200 , return_state=True , return_sequences=True )
decoder_outputs , _ , _ = decoder_lstm ( decoder_embedding , initial_state=encoder_states )
decoder_dense = tf.keras.layers.Dense( VOCAB_SIZE , activation=tf.keras.activations.softmax ) 
output = decoder_dense ( decoder_outputs )

model = tf.keras.models.Model([encoder_inputs, decoder_inputs], output )
model.compile(optimizer=tf.keras.optimizers.RMSprop(), loss='categorical_crossentropy')

model.summary()

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, None, 200)    378800      input_1[0][0]                    

The model will have Embedding, LSTM and Dense layers. The basic configuration is as follows.


*   2 Input Layers : One for `encoder_input_data` and another for `decoder_input_data`.
*   Embedding layer : For converting token vectors to fix sized dense vectors.
*   LSTM layer : Provide access to Long-Short Term cells.

Working : 

1.   The `encoder_input_data` comes in the Embedding layer (  `encoder_embedding` ). 
2.   The output of the Embedding layer goes to the LSTM cell which produces 2 state vectors ( `h` and `c` which are `encoder_states` )
3.   These states are set in the LSTM cell of the decoder.
4.   The decoder_input_data comes in through the Embedding layer.
5.   The Embeddings goes in LSTM cell ( which had the states ) to produce seqeunces.

In [7]:
# Start Training!
history = model.fit([encoder_input_data , decoder_input_data], decoder_output_data, batch_size=64, epochs=200) 
model.save( 'model.h5' ) 

Train on 564 samples
Epoch 1/200
564/564 [==============================] - 12s 21ms/sample - loss: 1.3390
Epoch 2/200
564/564 [==============================] - 2s 3ms/sample - loss: 1.1268
Epoch 3/200
564/564 [==============================] - 2s 3ms/sample - loss: 1.1022
Epoch 4/200
564/564 [==============================] - 2s 3ms/sample - loss: 1.0883
Epoch 5/200
564/564 [==============================] - 2s 3ms/sample - loss: 1.0735
Epoch 6/200
564/564 [==============================] - 2s 3ms/sample - loss: 1.0598
Epoch 7/200
564/564 [==============================] - 2s 3ms/sample - loss: 1.0466
Epoch 8/200
564/564 [==============================] - 2s 3ms/sample - loss: 1.0340
Epoch 9/200
564/564 [==============================] - 2s 3ms/sample - loss: 1.0222
Epoch 10/200
564/564 [==============================] - 2s 3ms/sample - loss: 1.0109
Epoch 11/200
564/564 [==============================] - 2s 3ms/sample - loss: 0.9986
Epoch 12/200
564/564 [=============================

In [14]:
history

## Defining inference models
We create inference models which help in predicting answers.

**Encoder inference model** : Takes the question as input and outputs LSTM states ( `h` and `c` ).

**Decoder inference model** : Takes in 2 inputs, one are the LSTM states ( Output of encoder model ), second are the answer input seqeunces ( ones not having the `<start>` tag ). It will output the answers for the question which we fed to the encoder model and its state values.

In [ ]:
def make_inference_models():
    
    encoder_model = tf.keras.models.Model(encoder_inputs, encoder_states)
    
    decoder_state_input_h = tf.keras.layers.Input(shape=( 200 ,))
    decoder_state_input_c = tf.keras.layers.Input(shape=( 200 ,))
    
    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
    
    decoder_outputs, state_h, state_c = decoder_lstm(
        decoder_embedding , initial_state=decoder_states_inputs)
    decoder_states = [state_h, state_c]
    decoder_outputs = decoder_dense(decoder_outputs)
    decoder_model = tf.keras.models.Model(
        [decoder_inputs] + decoder_states_inputs,
        [decoder_outputs] + decoder_states)
    
    return encoder_model , decoder_model

## Converse with ChatBot

1.   First, we take a question as input and predict the state values using `enc_model`.
2.   We set the state values in the decoder's LSTM.
3.   Then, we generate a sequence which contains the `<start>` element.
4.   We input this sequence in the `dec_model`.
5.   We replace the `<start>` element with the element which was predicted by the `dec_model` and update the state values.
6.   We carry out the above steps iteratively till we hit the `<end>` tag or the maximum answer length.

In [ ]:
def str_to_tokens( sentence : str ):
    words = sentence.lower().split()
    tokens_list = list()
    for word in words:
        tokens_list.append( tokenizer.word_index[ word ] ) 
    return preprocessing.sequence.pad_sequences( [tokens_list] , maxlen=maxlen_questions , padding='post')


In [10]:
enc_model , dec_model = make_inference_models()

for _ in range(10):
    states_values = enc_model.predict( str_to_tokens( input( 'Enter question : ' ) ) )
    empty_target_seq = np.zeros( ( 1 , 1 ) )
    empty_target_seq[0, 0] = tokenizer.word_index['start']
    stop_condition = False
    decoded_translation = ''
    while not stop_condition :
        dec_outputs , h , c = dec_model.predict([ empty_target_seq ] + states_values )
        sampled_word_index = np.argmax( dec_outputs[0, -1, :] )
        sampled_word = None
        for word , index in tokenizer.word_index.items() :
            if sampled_word_index == index :
                decoded_translation += ' {}'.format( word )
                sampled_word = word
        
        if sampled_word == 'end' or len(decoded_translation.split()) > maxlen_answers:
            stop_condition = True
            
        empty_target_seq = np.zeros( ( 1 , 1 ) )  
        empty_target_seq[ 0 , 0 ] = sampled_word_index
        states_values = [ h , c ] 

    print( decoded_translation )

Enter question : who invented computers
 it's a bit ambigous but british scientist charles babbage is regarded as the father of computers one might argue that john von neumann invented computers as we know them because he invented the princeton architecture in which instructions and data share the same memory field but are differentiated by context end
Enter question : what is operating system
 any software that supports python end
Enter question : which is better windows or macos
 it depends on which machine you're using to talk to me i'd prefer to not hurt your feelings linux always linux what are you trying to accomplish the os should support your goals end
Enter question : you are cruel
 i couldn't have said it better myself end
Enter question : ?


KeyError: ignored

## From the conversation:


Enter question : who invented computers <br>
 it's a bit ambigous but british scientist charles babbage is regarded as the father of computers one might argue that john von neumann invented computers as we know them because he invented the princeton architecture in which instructions and data share the same memory field but are differentiated by context end <br>
 <br>
Enter question : what is operating system <br>
 any software that supports python end <br>
 <br>
Enter question : which is better windows or macos <br>
 it depends on which machine you're using to talk to me i'd prefer to not hurt your feelings linux always linux what are you trying to accomplish the os should support your goals end <br>
 <br>
Enter question : you are cruel <br>
 i couldn't have said it better myself end
 <br>
 
Enter question : ?